In [43]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [44]:
raw = pd.read_csv('../data/DS0001/04690-0001-Data.tsv',sep='\t')
raw.shape

(3617, 4564)

## Dataset sampling

- Originally 3617 respondents
- Listwise deletion to 1640 participants
- Use dropna() function for Listwise deletion
- All of these participants had a spouse, a child, and friends when responding to the survey

- A1: Children = 2
- C7: Martial status = V410
- 

- V138: A1. Children in HH -> 1 No Child in HH && V201
- V401: C1. Marital Status -> 5 Never Marry
- V221: A3. Visit Friends -> 6 Never

In [45]:
# Married, have friends

sample = raw[(raw['V401'] == 1) & (raw['V221'] != 6) & (raw['V221'] != -96)]

In [46]:
no_child = sample[(sample['V138'] == 1) & (sample['V201'] == 5)]
print(no_child.shape)

(176, 4564)


In [47]:
df = pd.concat([sample, no_child, no_child]).drop_duplicates(keep=False)
print(df.shape)

(1718, 4564)


In [48]:
df.head()

V1  V3  V4    V5  V6  V7  V8         V9   V21  V33  V90  V91  V92  V93  \
1    2   5  28  1986   1   1   2  341202411  2040    1    2  -99    1  -99   
9   10   5  27  1986   5   1  10          0  5063   63    1    1    2  301   
11  12   5  22  1986   1   1  12  390801811  1164    1    2  -99    1  -99   
12  13   5  28  1986   1   1  13  390803511   485    1    2  -99    1  -99   
13  14   5  25  1986   5   1  14  390800611   484    1    2  -99    1  -99   

    V94  V95  V100  V101  V102  V103  V104  V105  V106  V107  V108  V109  \
1   -99  -99     2     4     1     1    44     2     2    42    11     1   
9     5    1     0     2     1     1    73     2     2    59   -99   -99   
11  -99  -99    12     7     1     1    47     2     2    46    11     1   
12  -99  -99    13     3     1     1    48     2     2    47    11     1   
13  -99  -99    14     5     1     2    55     2     2    55    41     1   

    V110  V111  V112  V113  V114  V115  V116  V117  V118  V119  V120  V121  \
1     15    12     2    17   -99   -99   -99   -99   -99   -99   -99   -99   
9    -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
11    22    11     1    20    12     2    18    12     2    18    11     1   
12     2   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
13     8    41     1    10    41     1    12   -99   -99   -99   -99   -99   

    V122  V123  V124  V125  V126  V127  V128  V129  V130  V131  V132  V133  \
1    -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
9    -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
11    17   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
12   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
13   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   

    V134  V135  V136  V137  V138  V201  V202  V203  V204  V205  V206  V207  \
1    -99   -99   -99   -99     5     5     0   -99   -99   -99   -99   -99   
9    -99   -99   -99   -99     1     1     3     1    48     2    51     2   
11   -99   -99   -99   -99     5     5     0   -99   -99   -99   -99   -99   
12   -99   -99   -99   -99     5     1     2     2    28     1    25   -99   
13   -99   -99   -99   -99     5     1     3     2    35     2    37     1   

    V208  V209  V210  V211  V212  V213  V214  V215  V216  V217  V218  V219  \
1    -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0   
9     44   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     8   
11   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0   
12   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     3   
13    29   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     8   

    V220  V221  V222  V223  V224  V225  V301  V302  V303  V304  V305  V306  \
1      2     2     3     2     2     2     2    19   -99   -99   -99   -99   
9      3     4     5     1     1     1     2    75    31   -99   -99   -99   
11     3     4     5     1     2     1     2    40    19   -99   -99   -99   
12     3     2     4     4     3     3     2    57    59   -99   -99   -99   
13     1     1     6     1     4     4     1    58    11     6   -99   -99   

    V307  V308  V309  V310  V311  V312  V313  V314  V315  V316  V317  V318  \
1     96   -99   -99   -99   -99     2     1     4     2     1     2     1   
9     96   -99   -99   -99   -99     2     3     2     3     1     1     1   
11    33    40   -99   -99   -99     2     2     2     2     1     1     1   
12     3    94   -99   -99   -99     2     1     4     3     1     4     2   
13    96   -99   -99   -99   -99     2     1     4     1     1     4     1   

    V319  V320  V321  V322  V323  V324  V325  V326  V327  V328  V329  V330  \
1      2     3     6     1     2     1     2   -99     1     1     4     4   
9      1     1     6     1     1     1     1     3     3     2     2     2   
11     1     2     8     1    

## Dependent Variables

Positive emotional state

- “In the past week: I felt happy” - H1E V1006
- “In the past week: I enjoyed life” - H1J V1010


Negative emotional state
- “In the past week: I felt sad” - H1M V1012
- “In the past week: I felt depressed” - H1A V1002
- “In the past week: I felt lonely” - H1F V1007

In [49]:
missing_val = {-95: np.nan, -96: np.nan, -99: np.nan}

df.replace({'V1006': missing_val, 'V1010': missing_val, 'V1012': missing_val, 
            'V1002': missing_val, 'V1007': missing_val}, inplace=True)

df['positive_emotion'] = df[['V1006', 'V1010']].mean(axis=1)
df['negative_emotion'] = df[['V1012', 'V1002', 'V1007']].mean(axis=1)

In [50]:
df.positive_emotion.value_counts()

3.0    1137
2.0     281
2.5     226
1.0      37
1.5      32
Name: positive_emotion, dtype: int64

In [51]:
df.negative_emotion.value_counts()

1.000000    881
1.333333    317
1.666667    243
2.000000    207
2.333333     39
2.666667     17
3.000000     11
Name: negative_emotion, dtype: int64

## Independent Variables

Spouse
- Love and care: C2 V405
- Willing to listen: C4 V407
- Too many demands: C3 V406
- Critical of you: C5 V408

Child
- Love and care: C14 V431
- Willing to listen: C16 V433
- Too many demands: C15 V432
- Critical of you: C17 V434

Friends
- Love and care: C47 V535
- Willing to listen: C49 V537
- Too many demands: C48 V536
- Critical of you: C50 V538

In [52]:
# Scale reverse-coded variables

score = {1: 5, 2: 4, 3: 3, 4: 2, 5: 1, -95: np.nan, -96: np.nan, -99: np.nan}

df = df.replace({'V405': score, 'V406': score, 'V407': score, 'V408': score,
                  'V431': score, 'V432': score, 'V433': score, 'V434': score,
                  'V535': score, 'V536': score, 'V537': score, 'V538': score})

df['spouse_support'] = df[['V405', 'V407']].mean(axis=1)
df['spouse_strain'] = df[['V406', 'V408']].mean(axis=1)
df['child_support'] = df[['V431', 'V433']].mean(axis=1)
df['child_strain'] = df[['V432', 'V434']].mean(axis=1)
df['friend_support'] = df[['V535', 'V537']].mean(axis=1)
df['friend_strain'] = df[['V536', 'V538']].mean(axis=1)

## Control Variables

- Confidants: C54 V546
- Age: A0C1 V104
- Income: R27 V1730
- Sex: AOB1 V103
- Education R13 V1646
    - Less than HS
    - HS diploma
    - College degree
    - 4-year degree
    - Graduate degree
- Retired: J1E V1105
- Age category
    - Age <45
    - Age 45-65
    - Age >65
- Number of Children
    - Children in HH: V105, V108, V111, V114, V117, V120, V123, V126, V129, V132, V135 -> Check each column 
    - Children elsewhere: V202



In [53]:
def age_categorize(row):  
    if row['age'] < 45:
        return 'under 45'
    elif row['age'] >= 45 and row['age'] <= 65:
        return '45-65'
    elif row['age'] > 65:
        return 'over 65'
    
def education_categorize(row):  
    if row['education'] <= 12 and row['V1647'] == 5:
        return '1' # Less than high school
    elif 12 <= row['education'] < 14 and row['V1648'] == 5:
        return '2' # High school diploma
    elif 14 <= row['education'] < 16 and row['V1648'] == 5:
        return '3' # College degree
    elif row['V1648'] == 1 and row['education'] < 17:
        return '4' # 4-year degree
    elif row['V1648'] == 1 and row['education'] >= 17:
        return '5' # Graduate degree
    return np.nan

In [54]:
children_set = [10, 11, 12, 13, 14]
family_member = ['V105', 'V108', 'V111', 'V114', 'V117', 'V120', 
                 'V123', 'V126', 'V129', 'V132', 'V135']

def count_children(row):
    child = 0
    for col in family_member:
        if row[col] in children_set:
            child += 1    
    return child


In [55]:
check_box = {1: 1, 5: 0}

df.replace({'V546': missing_val, 'V104': missing_val, 'V1730': missing_val,
            'V1105': check_box, 'V1646': missing_val, 'V202': missing_val,
            }, inplace=True)

df['confidants'] = df['V546']
df['age'] = df['V104']
df['age_group'] = df.apply(lambda row: age_categorize(row), axis=1)
df['income'] = df['V1730']
df['sex'] = df['V103']
df['education'] = df['V1646']
df['education_group'] = df.apply(lambda row: education_categorize(row), axis=1)
df['retired'] = df['V1105']
df['child_hh'] = df.apply(lambda row: count_children(row), axis=1)
df['child_elsewhere'] = df['V202'] 
df['num_child'] = df['child_hh'] + df['child_elsewhere']


In [56]:
no_child = df[df['num_child'] == 0]
df = pd.concat([df, no_child, no_child]).drop_duplicates(keep=False)
print(df.shape)

(1714, 4583)


In [57]:
data = df[['positive_emotion', 'negative_emotion', 'spouse_support', 'spouse_strain',
           'child_support', 'child_strain', 'friend_support', 'friend_strain',
           'confidants', 'age', 'age_group', 'income', 'sex', 'education', 'education_group',
           'retired', 'num_child']].copy()

print(data.shape)
data.head()

(1714, 17)


,positive_emotion,negative_emotion,spouse_support,spouse_strain,child_support,child_strain,friend_support,friend_strain,confidants,age,age_group,income,sex,education,education_group,retired,num_child
1,2.0,1.000000,4.5,3.5,3.5,3.5,2.0,2.0,1.0,44.0,under 45,9.0,1,12.0,NaN,0,2.0
9,3.0,1.666667,4.5,1.0,4.0,1.5,4.0,2.0,4.0,73.0,over 65,2.0,1,9.0,1,1,3.0
11,2.0,1.333333,4.0,1.5,4.0,1.5,3.5,2.5,3.0,47.0,45-65,7.0,1,10.0,1,0,5.0
12,3.0,1.000000,5.0,2.0,5.0,1.0,5.0,1.0,2.0,48.0,45-65,7.0,1,14.0,3,1,3.0
13,3.0,1.333333,4.5,1.5,5.0,1.0,3.5,3.0,7.0,55.0,45-65,7.0,2,5.0,1,0,3.0


In [58]:
data.education.value_counts()

12.0    578
14.0    175
16.0    131
17.0    128
13.0    122
8.0     119
11.0    105
10.0     97
15.0     63
9.0      56
7.0      43
6.0      34
5.0      16
3.0      13
4.0      10
2.0       7
1.0       7
0.0       6
Name: education, dtype: int64

In [59]:
data.education_group.value_counts()

1    467
3    226
4    132
5    124
2    120
Name: education_group, dtype: int64

## Statistical Hybrid Model

In [60]:
len(data)

1714

In [40]:
data[data['positive_emotion'].isnull()]

,positive_emotion,negative_emotion,spouse_support,spouse_strain,child_support,child_strain,friend_support,friend_strain,confidants,age,age_group,income,sex,education,education_group,retired,num_child
566,NaN,NaN,5.0,1.0,1.0,1.0,3.0,1.0,NaN,79.0,over 65,NaN,1,10.0,1,1,6.0
764,NaN,1.333333,3.5,1.0,5.0,2.0,4.5,2.0,7.0,71.0,over 65,3.0,2,8.0,1,0,2.0
1224,NaN,NaN,3.5,3.0,4.5,3.0,4.5,1.0,4.0,67.0,over 65,NaN,2,11.0,NaN,1,1.0
1977,NaN,1.333333,4.5,1.0,NaN,NaN,4.0,2.0,7.0,33.0,under 45,7.0,2,17.0,5,0,2.0
2276,NaN,NaN,4.0,1.0,5.0,1.0,4.5,1.0,1.0,61.0,45-65,10.0,2,12.0,NaN,0,2.0


In [41]:
from statsmodels.regression.mixed_linear_model import MixedLM

# For now, this is what we can do.
# In fact, this is what they do!
data = data.dropna()

# Describe variance components
vc = {'confidants': '0 + C(confidants)', 'age': '0 + C(age)', 'education_group': '0 + C(education_group)', 
      'income': '0 + C(income)', 'sex': '0 + C(sex)', 'retired': '0 + C(retired)', 'num_child': '0 + C(num_child)'} 

model1 = MixedLM.from_formula(
    'positive_emotion ~ spouse_support + spouse_strain + child_support + child_strain + friend_support + friend_strain',
    vc_formula = vc, data=data, groups=data['age_group'])

result1 = model1.fit()
result1.summary()


/Users/lucasrosenblatt/opt/miniconda3/envs/superquail_env/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/lucasrosenblatt/opt/miniconda3/envs/superquail_env/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/lucasrosenblatt/opt/miniconda3/envs/superquail_env/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/lucasrosenblatt/opt/miniconda3/envs/superquail_env/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/Users/lucasrosenblatt/opt/miniconda3

<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
==============================================================
Model:            MixedLM Dependent Variable: positive_emotion
No. Observations: 632     Method:             REML            
No. Groups:       3       Scale:              0.1981          
Min. group size:  101     Log-Likelihood:     -437.6914       
Max. group size:  317     Converged:          No              
Mean group size:  210.7                                       
--------------------------------------------------------------
                    Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept            2.345    0.275  8.520 0.000  1.806  2.885
spouse_support       0.042    0.025  1.681 0.093 -0.007  0.090
spouse_strain       -0.069    0.024 -2.887 0.004 -0.116 -0.022
child_support        0.031    0.025  1.269 0.204 -0.017  0.080
child_strain        -0.009    0.028 -0.316 0.752 -0.063  0.046
friend_support       0.081    0.024  3.352 0.001  0.034  0.128
friend_strain       -0.052    0.029 -1.782 0.075 -0.108  0.005
age Var              0.000    0.005                           
confidants Var       0.000    0.005                           
education_group Var  0.011                                    
income Var           0.023                                    
num_child Var        0.000    0.006                           
retired Var          0.127                                    
sex Var              0.124                                    
==============================================================

"""

In [ ]:
data['education_group'].hasnans

True

In [ ]:
data.education_group.describe

<bound method NDFrame.describe of 1       None
9          1
11         1
12         3
13         1
14         1
17      None
18      None
19      None
20         3
21      None
22      None
29         1
32      None
34         3
36         1
38      None
40      None
41         3
42         1
44         1
47         3
48         4
50         5
51      None
52         3
56         3
57         3
58      None
59         3
64         1
65      None
67         5
69         5
70         1
72      None
73         1
75         2
76         1
78      None
80         2
82      None
91      None
92         3
94         2
99         3
101        2
103        1
104        5
105        2
110        3
111        2
114        3
116        3
117        1
120     None
121        3
123        1
128        5
130     None
132     None
133        3
140        2
142     None
143        3
147        1
148     None
150     None
157        1
160     None
163        3
169        2
172        3
176        3
181 